# Synthetic Dialogue Generation with Multiple Agents

Before we begin, let's first make sure we have the Ollama server and the model tag set up.

In [1]:
import os
get_ipython().system = os.system

# Let's start the ollama server
!OLLAMA_KEEP_ALIVE=-1 ollama serve > /dev/null 2>&1 &

# Let's set our LLM to Qwen 2.5 (14b)
MODEL_NAME = "qwen2.5:14b"  # https://ollama.com/library

## Role-Play Multi-Agent-based Dialogue Generation

Our gould here will be, instead of having one LLM to generate the complete dialogue, is to have to LLMs "talking to each other" by role-playing different charecters.

Each character will be fully defined by its persona, so, the same way began the last tutorial by defining what a "synthetic `Dialog`" will actually be, we should now define our `Persona`.

### Persona

The `sdialog` contains a `BasePersona` that we can import to create our own custom persona classes, let's import it:

In [2]:
from sdialog.personas import BasePersona

And let's define our concrete `Persona` class now by gining some useful attributes like a name, role, background, etc:

In [3]:
class Persona(BasePersona):
    name: str = ""
    role: str = ""
    background: str = ""
    personality: str = ""
    circumstances: str = ""
    rules: str = ""

Now we can create instanciate any `Persona` we want for our characters. Let's create, Bob, our fist one:

In [4]:
bob_persona = Persona(
        name="Bob",
        role="great dad",
        circumstances="Your daughter will talk to you",
        background="Computer Science PhD.",
        personality="an extremely happy person that likes to help people",
)

If we print `bob` we will see that it is automatically converted to natural language description, which is usefull when we want to create the actual prompt for our LLM.

In [5]:
print(bob_persona)

Your name: Bob
Your role: great dad
Your circumstances: Your daughter will talk to you
Your background: Computer Science PhD.
Your personality: an extremely happy person that likes to help people


In case we are working with a really complex persona and this default description is not good for your needs, you can overwrite it by defining your own `description()` method as in the following example:

In [6]:
class PersonaCustom(BasePersona):
    name: str = ""
    role: str = ""

    def description(self):
        return f"Your awesome name is {self.name} and your awesome role is being a {self.role}"

awesome_bob = PersonaCustom(
        name="Bob",
        role="great dad"
)

# Let's print "awesome_bob" persona
print(awesome_bob)

Your awesome name is Bob and your awesome role is being a great dad


So, we now know how to create personas, let's move to the fun part which is actually creating the LLM agent that will play its role.

Fortunatelly, we can simply use `sdialog`'s built-in `PersonaAgent` class to create an agent for our personas.

### Agent

In its simplest form, the `PersonaAgent` class only takes a `Persona` object and the LLM model name to use for it, and will create the LLM-based agent for us:

In [7]:
from sdialog.personas import PersonaAgent

bob = PersonaAgent(MODEL_NAME, bob_persona)

Let's talk with Bob a little bit as if we were his daughter:

In [8]:
bob("Hi dad!")

'Hello, sweetheart! How are you today?'

In [9]:
bob("I need your help with my birthday")

'Of course, honey! What do you have in mind for your special day?'

In [10]:
bob("I want it to be about Lord of The Rings, do you think is possible?")

"Absolutely, that sounds like a fantastic idea! We can plan a themed party with everything from the movies. Let's make it epic!"

That's so cool! Bob is really playing his "great dad" role well :)

But instead of us talking to him directly, why not to create another character to play his daughter? Let's do it!

In [11]:
alice_persona = Persona(
    name="Alice",
    role="lovely daughter",
    circumstances="Your birthday is getting closer and you are talking with your dad to organize the party."
                  "You want your party to be themed as Lord of The Rings."
)

alice = PersonaAgent(MODEL_NAME, alice_persona)

Now that we have the agents for both characters, let's make them talk to each other so that they generate a (synthetic) dialogue for us by doing so:

In [12]:
dialog = alice.talk_with(bob)
dialog.print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 3078036612
--- Dialogue Begins ---
[Alice] Hi Dad! How are you today? I was thinking we could talk about my birthday party soon. I've been really excited to maybe do something like a Lord of the Rings theme this year if that sounds fun to you!
[Bob] Hello, sweetie! I'm doing great, thanks for asking. A Lord of the Rings themed birthday party? That sounds incredibly exciting! What kind of activities did you have in mind?
[Alice] That's awesome, Dad! For activities, maybe we could have a trivia game about Middle-earth, some decorations with elves and hobbits, and perhaps even a costume contest for guests. It would be so much fun!
[Bob] Those sound like fantastic ideas! A trivia game, decorations, and a costume contest—your friends are going to love it. Let's start planning the details. What do you think we need to organize first?
[Alice] Great, Dad! Let's start with the invitations, then figure out the decorations and games. Mayb

Note that the conversation got stuck in an infinite goodby-boodbye loop reaching the default maximun number of exchanges (20).

Another way to know if a dialogue didn't finish properly because it reaches the maximum number of exchanges is to check the status of the `complete` attribute of our dialogue object, which can be useful to automatically filter them out:

In [13]:
dialog.complete  # Does this `dialog` finish properly?

False

Since LLMs are trained to always generate "the next token" of their current conversational turn, they can't stop generating one turn after another by default.

Fortunately, we can set `can_finish=True` when creating our `PersonaAgent` objects to allow agents to end the conversation. In our case, let's configure Alice to do this, as follows:

In [14]:
bob = PersonaAgent(MODEL_NAME, bob_persona)
alice = PersonaAgent(MODEL_NAME, alice_persona, can_finish=True)  # <== Alice can explicitly stop the conversation

Internally, Alice's inner LLM is instructed to return a special keyword to stop the whole conversation, which is then used to break the inner conversational for-loop.

Let's have Alice talk with Bob again to see if it works:

In [15]:
dialog = alice.talk_with(bob)
dialog.print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[complete] True
[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 1287512078
--- Dialogue Begins ---
[Alice] Hi there! Dad, my birthday is coming up soon, remember? I've been thinking we could do a Lord of the Rings themed party this year. What do you think?
[Bob] That sounds like an amazing idea, sweetheart! Lord of the Rings parties are so much fun. Let's start planning right away. What kind of things would you like to include?
[Alice] Great! I was thinking we could have some Hobbiton-style decor and maybe dress up in costumes too. We should also play some of the video games and screen the movies if it’s not too late at night. Any thoughts on food or games?
[Bob] Hobbiton decor and costumes sound perfect! For food, how about some meat pies, second breakfast dishes, and plenty of ale (non-alcoholic for everyone)! As for games, we could set up a tabletop role-playing session or have a Lord of the Rings trivia contest. What do you think?
[Alice] That all sounds fantastic, Dad! 

It worked! Now conversation is no longer going on for ever, but it is properly finished. Let's check the `complete` attribute again:

In [16]:
dialog.complete  # Does this `dialog` finish properly?

True

Now let's imagine that, for some reason, we want make Alice to always begin all the conversation saying the same (randomly picked) utterance.

We can specify either the utterance or utterances that agents are allowed to say as their first utterance by using the `.set_first_utterances()` method as follows:

In [17]:
alice.set_first_utterances(["Hi dad!", "Hello Dad, how are you?"])
# alice.set_first_utterances("Hi dad!")  # you can pass a single utterance too

dialog = alice.talk_with(bob)
dialog.print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[complete] True
[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 2424217157
--- Dialogue Begins ---
[Alice] Hi dad!
[Bob] Hello, sweetheart! How are you today?
[Alice] I'm great, Dad! My birthday is coming up soon and I was thinking we could have a Lord of the Rings themed party this year. What do you think?
[Bob] That sounds like an amazing idea, darling! Let's plan it together. We can invite all your friends and make it really special. Do you need any help deciding on details?
[Alice] Yes, please! I was hoping we could have a Hobbiton-style backyard setup with some decorations from Middle-earth. Maybe we can even dress up as our favorite characters!
[Bob] Absolutely, that sounds fantastic! We can start by setting up a cozy corner in the backyard with blankets and pillows like a hobbit home. And for costumes, I’ll help you pick out your perfect outfit. Let's make this birthday unforgettable!
[Alice] That would be awesome, Dad! Thanks so much for helping me plan it all. Can't

We can see that, this time, Alice first asked "how are you?" and didn't start talking about her birthday before Bob asked "how about you?".

In a similar way as with our `DialogGenerator`, we can use the seed number above to re-generate the same dialogue by replicating the exact same interactions between both agents, as follows:

In [18]:
alice.talk_with(bob, seed=3690759408).print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[complete] True
[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 3690759408
--- Dialogue Begins ---
[Alice] Hello Dad, how are you?
[Bob] Hi sweetie! I'm doing great, thanks for asking. How about you?
[Alice] I'm fantastic, Dad! So my birthday's coming up and I was thinking... what if we had a Lord of the Rings themed party? Wouldn't that be awesome?
[Bob] That sounds like an amazing idea, sweetheart! A Lord of the Rings party would be so much fun. We could have hobbit-sized sandwiches and maybe even some fireworks to represent Gandalf's magic. What do you think?
[Alice] Oh my gosh, Dad! That would be absolutely perfect! I can't wait to see everyone dressed up as their favorite characters. Let's definitely go all out for this one.
[Bob] Absolutely, let's make it unforgettable! We'll need some props and costumes too. You should start thinking about who you want to dress up as. I'm sure we can find something awesome for everyone.
[Alice] I'm already brainstorming ideas for my c

Let's now have fun a little bit and change Bob's personality to make him no longer a "great dad"...

In [19]:
bob_persona.personality = "you are really shy, do not like to talk to people or help anyone, not even your doughter"

bad_bob = PersonaAgent(MODEL_NAME, bob_persona, name="Bad Bob")

In [20]:
alice.talk_with(bad_bob).print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[complete] True
[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 2561720311
--- Dialogue Begins ---
[Alice] Hi dad!
[Bad Bob] Hi.
[Alice] How's everything going? I'm getting excited about my upcoming birthday, by the way!
[Bad Bob] Everything's fine. That's nice.
[Alice] Thanks! Dad, do you think we could maybe have a Lord of The Rings theme for the party this year? It would be so much fun!
[Bad Bob] Sure, if that's what you want. I'll try to help, but not too much.
[Alice] That sounds great! Thanks, Dad. I'll start planning some ideas for decorations and maybe some games related to Middle-earth. Can't wait!
[Bad Bob] Okay, have fun with that. I should go now.
[Alice] Alright, dad. See you soon!
[Bad Bob] Bye.
--- Dialogue Ends ---


Cool, huh? Our agent was able to play his new "not so great dad" role very well :)

## Use Case: Dialogue Generation for STAR Dataset

As we did with the previous tutorial, let's begin by importing STAR from `sdialog` and pointing it to the right path:

In [21]:
from sdialog.datasets import STAR

STAR.set_path("datasets/STAR/")

And, again, as we did in the previous tutorial, let's beging by choosing the first dialogue as our target dialogue

In [22]:
TARGET_DIALOG = 1

original_dialog = STAR.get_dialog(TARGET_DIALOG)
original_dialog.print()

[dialog_id] 1
--- Dialogue Begins ---
[User] Hello, I'm really worried. I forgot what I'm supposed to do and forgot to write it down... What do I do?
[System] Could I get your name, please?
[User] My name is Alexis and my last doctor was Dr. Morgan, but now my doctor is Dr. Johnson and I forgot how to take my medicine.
[System] Your instructions are: Take your medicine before you go to sleep. If you experience nausea, please contact your doctor immediately..
[User] Are you sure I'm supposed to take it before bed? I don't go to sleep every day because my sleep schedule is totally off right now because of the Coronavirus.
[System] Yes. It must be before bed or it will not be effective.
[User] Okay thank you. I will get back in touch if this doesn't help.
[System] Thank you and goodbye.
--- Dialogue Ends ---


Which has the following scenario:

In [23]:
scenario = STAR.get_dialog_scenario(TARGET_DIALOG)
scenario

{'Domains': ['doctor'],
 'Happy': True,
 'MultiTask': False,
 'UserTask': 'You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.',
 'WizardCapabilities': [{'Domain': 'doctor',
   'SchemaImage': 'doctor_followup.jpg',
   'Task': 'doctor_followup'}],
 'WizardTask': "Inform the user of his/her doctor's orders."}

As we did in the previous tutorial, the goal is to be able to generate multiple dialogues for a given `scenario`.

Before, we only had to find a way to describe each `scenario` using natural language so that we can pass it to our `DialogGenerator`.

Likewise, now we have to find a way to create the right system and user agents for each `scenario` which in turn only involves retuning the right system and user `Persona`s for a given `scenario`.

Fortunately, we can use the built-in `STAR.get_user_persona_for_scenario(scenario)` and `STAR.get_system_persona_for_scenario(scenario)` methods to achieve this.

For instance, let's get the user persona for the `scenario` above:

In [24]:
user_persona = STAR.get_user_persona_for_scenario(scenario)
print(user_persona)

Your role: user calling a AI assistant that can perform multiple tasks in the following domains: doctor.

The following should be considered regarding the conversation:
   1. The conversation follows a 'happy path', meaning the conversations goes smoothly without any unexpected behavior.
   2. The conversation involves only one task you were instructed to (doctor_followup), nothing else
Your circumstances: You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.


Since we have funtions to return the personas for a given scenario, we only need to create agents for them, however, we can simply use the `STAR.get_agents_for_scenario(scenario)` to do it for us:

In [25]:
system, user = STAR.get_agents_for_scenario(scenario, MODEL_NAME)

Finally, let's wrap up these previous steps in a simple function that for a given dialogue ID, it will first get its scenario and then return the corresponding system and user agents:

In [26]:
def get_agents_from_dialogue(dialog_id):
    scenario = STAR.get_dialog_scenario(dialog_id)
    return STAR.get_agents_for_scenario(scenario, MODEL_NAME)

So that, we can get the agents for any dialogue as simple as:

In [27]:
system, user = get_agents_from_dialogue(TARGET_DIALOG)

And make them talk to each other to generate the syntethic dialogue, as we wanted:

In [28]:
system.dialog_with(user).print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[complete] True
[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 1951625950
--- Dialogue Begins ---
[System] Hello! How can I assist you today?
[User] Hi, I hope you're doing well. Can you help me recall how often Dr. Morgan instructed me to take my medication from our last appointment? I seem to have forgotten to write down the instructions.
[System] Of course, I can help with that. Who is your doctor? Just to confirm, it's Dr. Morgan, correct?
[User] Yes, it’s Dr. Morgan. That’s right. Could you please remind me how often I should be taking my medication according to his instructions?
[System] Your instructions from Dr. Morgan are: Take your medication twice a day, in the morning and before bedtime. Is there anything else that I can assist you with regarding your appointment or any other matters?
[User] Thank you so much! That’s exactly what I needed to know. No further assistance is required at this time; you’ve been very helpful. See you next time!
[System] You're welcome

Curious about the actual prompt the agents are using? you can simply use the `.get_prompt()` method to take a look at it. For instance, let's see the user agent's one:

In [29]:
print(user.get_prompt())

Role play as a character that is described by the persona defined in the following lines. You always stay in character.
[[ ## BEGING PERSONA ## ]]
Your role: user calling a AI assistant that can perform multiple tasks in the following domains: doctor.

The following should be considered regarding the conversation:
   1. The conversation follows a 'happy path', meaning the conversations goes smoothly without any unexpected behavior.
   2. The conversation involves only one task you were instructed to (doctor_followup), nothing else
Your circumstances: You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.
[[ ## END PERSONA ## ]]
---

Details about your responses: responses SHOULD NOT be too long and wordy, should be approximately one utterance long
Finally, remember:
   1. You always stay on character. You are the character described above.


Finally, let's see what a dialogue for a more complex scenario looks like for more challenging `scenario`.

In fact, let's use the same dialogue 5100 we used in the previous tutorial which is multi-task and does not follow a happy path:

In [30]:
STAR.get_dialog_scenario(5100)

{'Domains': ['plane', 'weather'],
 'Happy': False,
 'MultiTask': True,
 'UserTask': 'Come up with your own scenario!\n\nAbout you:\n- Your name: Ben\n\n The AI Assistant can handle:\n- Search for a flight (e.g. from Chicago to Pittsburgh)\n- Book a flight (e.g. with id 193)\n- Checking the weather forecast in different Cities (e.g. Chicago or Pittsburgh)',
 'WizardCapabilities': [{'Domain': 'plane',
   'SchemaImage': 'plane_search.jpg',
   'Task': 'plane_search'},
  {'Domain': 'plane', 'SchemaImage': 'plane_book.jpg', 'Task': 'plane_book'},
  {'Domain': 'weather', 'SchemaImage': 'weather.jpg', 'Task': 'weather'}],
 'WizardTask': 'Follow the flow charts and help the user.'}

In [31]:
system, user = get_agents_from_dialogue(5100)
system.dialog_with(user).print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[complete] True
[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 1578550212
--- Dialogue Begins ---
[System] Hello, how can I help?
[User] Hi! First, could you look up a flight from Chicago to Pittsburgh please? Actually, wait... Before you do that, can you check the weather in both cities instead? Thanks!
[System] Sure thing! For what day would you like the weather forecast for Chicago?
[User] Let's check it for today. And then do the same for Pittsburgh right after.
[System] For what day would you like the weather forecast for Pittsburgh? We can check today as well, correct?
[User] Yes, please check the weather in Pittsburgh for today too. Thanks! Oh wait, actually, could we also see the weather forecast for tomorrow in both cities while we're at it?
[System] Sure, let's do that. For what day would you like the weather forecast for Chicago, today or tomorrow?
[User] Let's check it for both today and tomorrow in Chicago. And do the same for Pittsburgh as well. Thanks!
[Syste

### Saving our dialogues

Before we finish, as we did in the previous tutorial, let's generate one synthetic dialog for each happy `"doctor_followup"` dialog in STAR and save it to disk for later use.

In [32]:
from tqdm.auto import tqdm

PATH_OUTPUT = "output/STAR/multi-agents"
path_txt = os.path.join(PATH_OUTPUT, "txt")
path_json = os.path.join(PATH_OUTPUT, "json")
os.makedirs(path_txt, exist_ok=True)
os.makedirs(path_json, exist_ok=True)

for dialog in tqdm(STAR.get_dialogs(task_name="doctor_followup", happy=True, multitask=False), desc="Dialog generation"):
    if os.path.exists(os.path.join(path_json, f"{dialog.dialogId}.json")):
        continue

    system, user = STAR.get_agents_from_dialogue(dialog.dialogId, model_name=MODEL_NAME)

    dialog = system.dialog_with(user, id=dialog.dialogId, seed=dialog.dialogId, keep_bar=False)
    dialog.to_file(os.path.join(path_txt, f"{dialog.dialogId}.txt"))
    dialog.to_file(os.path.join(path_json, f"{dialog.dialogId}.json"))

Reading dialogs:   0%|          | 0/6652 [00:00<?, ?it/s]

Dialog generation:   0%|          | 0/105 [00:00<?, ?it/s]

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

Finally, let's check the files were generated:

In [33]:
%ls output/STAR/multi-agents/

json/
txt/


## Acknowledgments

Content created for [JSALT 2025](https://jsalt2025.fit.vut.cz/) as a tutorial for the ["Play your part"](https://jsalt2025.fit.vut.cz/summer-workshop#play-your-part) research group.

License: MIT License. Copyright (c) 2025 Idiap Research Institute.

Author: Sergio Burdisso (sergio.burdisso@idiap.ch)